In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 코랩에 Mecah 설치
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
!cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab_light_220429.sh

fatal: destination path 'Mecab-ko-for-Google-Colab' already exists and is not an empty directory.
bash: install_mecab-ko_on_colab_light_220429.sh: No such file or directory


In [ ]:
!pip3 install mecab-python

In [ ]:
from konlpy.tag import Mecab
mecab = Mecab()

hgtk : https://github.com/bluedisk/hangul-toolkit

In [ ]:
# 한글 자모 단위 처리 패키지 설치
!pip install hgtk

In [ ]:
# fasttext 설치
!git clone https://github.com/facebookresearch/fastText.git
%cd fastText
!make
!pip install .

Cloning into 'fastText'...
remote: Enumerating objects: 3946, done.
remote: Counting objects: 100% (980/980), done.
remote: Compressing objects: 100% (151/151), done.
remote: Total 3946 (delta 878), reused 839 (delta 828), pack-reused 2966
Receiving objects: 100% (3946/3946), 8.25 MiB | 20.22 MiB/s, done.
Resolving deltas: 100% (2510/2510), done.
/content/Mecab-ko-for-Google-Colab/fastText
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/args.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/autotune.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/matrix.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/dictionary.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/loss.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/productquantizer.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/densematrix.cc
c++ -pthr

# 1. 데이터 로드

In [ ]:
import re
import pandas as pd
import urllib.request
from tqdm import tqdm
import hgtk
from konlpy.tag import Mecab

In [ ]:
# 네이버 쇼핑 리뷰
urllib.request.urlretrieve("https://raw.githubusercontent.com/bab2min/corpus/master/sentiment/naver_shopping.txt", filename="ratings_total.txt")

('ratings_total.txt', <http.client.HTTPMessage at 0x7fb2af1af4c0>)

In [ ]:
total_data = pd.read_table('ratings_total.txt', names=['ratings', 'reviews'])
print('전체 리뷰 개수 :',len(total_data)) # 전체 리뷰 개수 출력

전체 리뷰 개수 : 200000


In [ ]:
total_data[:5]

,ratings,reviews
0,5,배공빠르고 굿
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...
4,5,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ


# 2. hgtk 튜토리얼

In [ ]:
# 한글인지 체크
hgtk.checker.is_hangul('ㄱ')

True

In [ ]:
# 한글인지 체크
hgtk.checker.is_hangul('28')

False

In [ ]:
# 음절을 초성, 중성, 종성으로 분해
hgtk.letter.decompose('남')

('ㄴ', 'ㅏ', 'ㅁ')

In [ ]:
# 초성, 중성을 결합
hgtk.letter.compose('ㄴ', 'ㅏ')

'나'

In [ ]:
# 초성, 중성, 종성을 결합
hgtk.letter.compose('ㄴ', 'ㅏ', 'ㅁ')

'남'

In [ ]:
# 한글이 아닌 입력에 대해서는 에러 발생.
hgtk.letter.decompose('1')

NotHangulException: ignored

In [ ]:
# 결합할 수 없는 상황에서는 에러 발생
hgtk.letter.compose('ㄴ', 'ㅁ', 'ㅁ')

NotHangulException: ignored

# 3. 데이터 전처리

In [ ]:
def word_to_jamo(token):
  def to_special_token(jamo):
    if not jamo:
      return '-'
    else:
      return jamo

  decomposed_token = ''
  for char in token:
    try:
      # char(음절)을 초성, 중성, 종성으로 분리
      cho, jung, jong = hgtk.letter.decompose(char)

      # 자모가 빈 문자일 경우 특수문자 -로 대체
      cho = to_special_token(cho)
      jung = to_special_token(jung)
      jong = to_special_token(jong)
      decomposed_token = decomposed_token + cho + jung + jong

    # 만약 char(음절)이 한글이 아닐 경우 자모를 나누지 않고 추가
    except Exception as exception:
      if type(exception).__name__ == 'NotHangulException':
        decomposed_token += char

  # 단어 토큰의 자모 단위 분리 결과를 추가
  return decomposed_token

In [ ]:
word_to_jamo('남동생')

'ㄴㅏㅁㄷㅗㅇㅅㅐㅇ'

In [ ]:
word_to_jamo('여동생')

'ㅇㅕ-ㄷㅗㅇㅅㅐㅇ'

In [ ]:
mecab = Mecab()

In [ ]:
print(mecab.morphs('선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다.'))

['선물', '용', '으로', '빨리', '받', '아서', '전달', '했어야', '하', '는', '상품', '이', '었', '는데', '머그', '컵', '만', '와서', '당황', '했', '습니다', '.']


In [ ]:
def tokenize_by_jamo(s):
    return [word_to_jamo(token) for token in mecab.morphs(s)]

In [ ]:
print(tokenize_by_jamo('선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다.'))

['ㅅㅓㄴㅁㅜㄹ', 'ㅇㅛㅇ', 'ㅇㅡ-ㄹㅗ-', 'ㅃㅏㄹㄹㅣ-', 'ㅂㅏㄷ', 'ㅇㅏ-ㅅㅓ-', 'ㅈㅓㄴㄷㅏㄹ', 'ㅎㅐㅆㅇㅓ-ㅇㅑ-', 'ㅎㅏ-', 'ㄴㅡㄴ', 'ㅅㅏㅇㅍㅜㅁ', 'ㅇㅣ-', 'ㅇㅓㅆ', 'ㄴㅡㄴㄷㅔ-', 'ㅁㅓ-ㄱㅡ-', 'ㅋㅓㅂ', 'ㅁㅏㄴ', 'ㅇㅘ-ㅅㅓ-', 'ㄷㅏㅇㅎㅘㅇ', 'ㅎㅐㅆ', 'ㅅㅡㅂㄴㅣ-ㄷㅏ-', '.']


In [ ]:
tokenized_data = []

for sample in tqdm(total_data['reviews'].to_list()):
    tokenzied_sample = tokenize_by_jamo(sample) # 자소 단위 토큰화
    tokenized_data.append(tokenzied_sample)

100%|██████████| 200000/200000 [00:49<00:00, 4053.68it/s]


In [ ]:
len(tokenized_data)

200000

In [ ]:
tokenized_data[0]

['ㅂㅐ-ㄱㅗㅇ', 'ㅃㅏ-ㄹㅡ-', 'ㄱㅗ-', 'ㄱㅜㅅ']

In [ ]:
def jamo_to_word(jamo_sequence):
  tokenized_jamo = []
  index = 0

  # 1. 초기 입력
  # jamo_sequence = 'ㄴㅏㅁㄷㅗㅇㅅㅐㅇ'

  while index < len(jamo_sequence):
    # 문자가 한글(정상적인 자모)이 아닐 경우
    if not hgtk.checker.is_hangul(jamo_sequence[index]):
      tokenized_jamo.append(jamo_sequence[index])
      index = index + 1

    # 문자가 정상적인 자모라면 초성, 중성, 종성을 하나의 토큰으로 간주.
    else:
      tokenized_jamo.append(jamo_sequence[index:index + 3])
      index = index + 3

  # 2. 자모 단위 토큰화 완료
  # tokenized_jamo : ['ㄴㅏㅁ', 'ㄷㅗㅇ', 'ㅅㅐㅇ']

  word = ''
  try:
    for jamo in tokenized_jamo:

      # 초성, 중성, 종성의 묶음으로 추정되는 경우
      if len(jamo) == 3:
        if jamo[2] == "-":
          # 종성이 존재하지 않는 경우
          word = word + hgtk.letter.compose(jamo[0], jamo[1])
        else:
          # 종성이 존재하는 경우
          word = word + hgtk.letter.compose(jamo[0], jamo[1], jamo[2])
      # 한글이 아닌 경우
      else:
        word = word + jamo

  # 복원 중(hgtk.letter.compose) 에러 발생 시 초기 입력 리턴.
  # 복원이 불가능한 경우 예시) 'ㄴ!ㅁㄷㅗㅇㅅㅐㅇ'
  except Exception as exception:
    if type(exception).__name__ == 'NotHangulException':
      return jamo_sequence

  # 3. 단어로 복원 완료
  # word : '남동생'

  return word

In [ ]:
jamo_to_word('ㄴㅏㅁㄷㅗㅇㅅㅐㅇ')

'남동생'

# 4. FastText

In [ ]:
import fasttext

In [ ]:
with open('tokenized_data.txt', 'w') as out:
  for line in tqdm(tokenized_data, unit=' line'):
    out.write(' '.join(line) + '\n')

100%|██████████| 200000/200000 [00:00<00:00, 368184.62 line/s]


In [ ]:
model = fasttext.train_unsupervised('tokenized_data.txt', model='cbow')

In [ ]:
model.save_model("fasttext.bin")

In [ ]:
model = fasttext.load_model("fasttext.bin")

In [ ]:
model[word_to_jamo('남동생')]   

array([ 0.4549458 ,  0.90374774,  0.04658839, -0.06666744, -0.5707408 ,
        0.15362757,  0.34270838,  0.8030201 , -0.5554203 , -0.6724852 ,
        0.2697398 ,  0.35657617,  0.09884271,  1.2222209 , -1.0018486 ,
       -0.07693717,  0.06426756,  0.47788393, -0.32691067, -0.00349128,
        0.46675646, -0.07882056,  0.09125831, -0.35830528, -0.02352133,
       -0.6972543 ,  0.8375617 ,  0.9449744 ,  0.30783752, -0.55533063,
        0.61867666, -0.22547989, -0.0624616 , -0.37748334,  0.7547899 ,
       -0.00745   , -0.03077007, -0.3424948 ,  0.0877356 , -0.16679713,
        0.9232839 ,  0.7421618 , -0.2586986 ,  0.16082908, -1.0991035 ,
       -0.8474681 , -0.2841582 ,  0.21239513,  0.47977424, -0.2946133 ,
        0.16922358,  0.02416898,  0.7255279 , -0.66462356, -0.79118454,
        0.75241196, -0.54106903,  0.12149671, -0.30856   , -0.38153017,
       -0.64430434, -0.867574  , -0.35850525,  0.5695654 ,  0.7113107 ,
        0.1984201 ,  0.6968861 , -0.19956519,  0.22699238,  0.38

In [ ]:
model.get_nearest_neighbors(word_to_jamo('남동생'), k=10)

[(0.893354594707489, 'ㄷㅗㅇㅅㅐㅇ'),
 (0.8385293483734131, 'ㄴㅏㅁㅊㅣㄴ'),
 (0.7786555290222168, 'ㅊㅣㄴㄱㅜ-'),
 (0.7726092338562012, 'ㄴㅏㅁㅍㅕㄴ'),
 (0.7439638376235962, 'ㅈㅗ-ㅋㅏ-'),
 (0.7224093675613403, 'ㄴㅏㄴㅅㅐㅇ'),
 (0.709885835647583, 'ㄴㅏㅁㅇㅏ-'),
 (0.6998860239982605, 'ㅇㅓㄴㄴㅣ-'),
 (0.6939324736595154, 'ㅅㅐㅇㅇㅣㄹ'),
 (0.6909884810447693, 'ㅅㅣㄴㄹㅏㅇ')]

In [ ]:
def transform(word_sequence):
  return [(jamo_to_word(word), similarity) for (similarity, word) in word_sequence]

In [ ]:
print(transform(model.get_nearest_neighbors(word_to_jamo('남동생'), k=10)))

[('동생', 0.893354594707489), ('남친', 0.8385293483734131), ('친구', 0.7786555290222168), ('남편', 0.7726092338562012), ('조카', 0.7439638376235962), ('난생', 0.7224093675613403), ('남아', 0.709885835647583), ('언니', 0.6998860239982605), ('생일', 0.6939324736595154), ('신랑', 0.6909884810447693)]


# 5. Word2Vec

In [ ]:
# 불용어 정의
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']


In [ ]:
tokenized_data2 = []
for sentence in tqdm(total_data['reviews'].to_list()):
    tokenized_sentence = mecab.morphs(sentence) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence 
                                  if not word in stopwords] # 불용어 제거
    tokenized_data2.append(stopwords_removed_sentence)

100%|██████████| 200000/200000 [00:19<00:00, 10151.34it/s]


In [ ]:
from gensim.models import Word2Vec

model2 = Word2Vec(sentences = tokenized_data2, vector_size = 1000, window = 5, min_count = 5, workers = 4, sg = 0)

In [ ]:
# 완성된 임베딩 매트릭스의 크기 확인
model2.wv.vectors.shape

(15005, 1000)

# 6. FastText Word2Vec 결과 비교

In [ ]:
## FastText
print(transform(model.get_nearest_neighbors(word_to_jamo('남동생'), k=10)))

[('동생', 0.893354594707489), ('남친', 0.8385293483734131), ('친구', 0.7786555290222168), ('남편', 0.7726092338562012), ('조카', 0.7439638376235962), ('난생', 0.7224093675613403), ('남아', 0.709885835647583), ('언니', 0.6998860239982605), ('생일', 0.6939324736595154), ('신랑', 0.6909884810447693)]


In [ ]:
## Word2Vec
print(model2.wv.most_similar("남동생"))

[('앞둔', 0.7094454765319824), ('양가', 0.6974008679389954), ('사촌', 0.6916664838790894), ('기념일', 0.6876041889190674), ('후배', 0.6875878572463989), ('어린이날', 0.6856410503387451), ('계신', 0.6764802932739258), ('시어머님', 0.6724096536636353), ('01', 0.6679489016532898), ('졸업', 0.6663628816604614)]
